In [4]:
import csv
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, XSD

def create_ontology(csv_file, base_url):
    # Create a new RDF graph
    g = Graph()

    # Define namespaces
    ns = Namespace(base_url)
    g.bind("ns", ns)

    # Create a class for Equipment Efficiency
    equipment_efficiency = URIRef(ns["EquipmentEfficiency"])
    g.add((equipment_efficiency, RDF.type, RDFS.Class))
    g.add((equipment_efficiency, RDFS.label, Literal("Equipment Efficiency", lang="en")))

    # Read the CSV file
    with open(csv_file, 'r') as file:
        csv_reader = csv.reader(file)
        headers = next(csv_reader)
        efficiency_values = next(csv_reader)[1:]  # Skip the 'efficiency' column name

    # Process each column (except the first 'equipment' column)
    for i, header in enumerate(headers[1:], start=1):
        parts = header.split()
        equipment = "_".join(parts[:-1])  # Join all parts except the last one
        material = parts[-1]  # Last part is the material

        # Create URI for this specific equipment-material combination
        uri = URIRef(ns[f"{equipment}_{material}"])

        # Add triples
        g.add((uri, RDF.type, equipment_efficiency))
        g.add((uri, RDFS.label, Literal(f"Efficiency of {equipment} for {material}", lang="en")))
        g.add((uri, ns.hasEquipment, Literal(equipment)))
        g.add((uri, ns.hasMaterial, Literal(material)))
        g.add((uri, ns.hasEfficiency, Literal(efficiency_values[i-1], datatype=XSD.float)))

    return g

# Usage
csv_file = "corrected_mrf_equipment_efficiency.csv"
base_url = "http://example.org/mrf-equipment/"

ontology = create_ontology(csv_file, base_url)
ontology.serialize()

'@prefix ns: <http://example.org/mrf-equipment/> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\nns:EquipmentEfficiency a rdfs:Class ;\n    rdfs:label "Equipment Efficiency"@en .\n\nns:disc_screen1_aluminum a ns:EquipmentEfficiency ;\n    rdfs:label "Efficiency of disc_screen1 for aluminum"@en ;\n    ns:hasEfficiency "0.0"^^xsd:float ;\n    ns:hasEquipment "disc_screen1" ;\n    ns:hasMaterial "aluminum" .\n\nns:disc_screen1_cardboard a ns:EquipmentEfficiency ;\n    rdfs:label "Efficiency of disc_screen1 for cardboard"@en ;\n    ns:hasEfficiency "0.0"^^xsd:float ;\n    ns:hasEquipment "disc_screen1" ;\n    ns:hasMaterial "cardboard" .\n\nns:disc_screen1_film a ns:EquipmentEfficiency ;\n    rdfs:label "Efficiency of disc_screen1 for film"@en ;\n    ns:hasEfficiency "0.0"^^xsd:float ;\n    ns:hasEquipment "disc_screen1" ;\n    ns:hasMaterial "film" .\n\nns:disc_screen1_glass a ns:EquipmentEfficiency ;\n    rdfs:label "Effic

In [11]:
import csv
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, SKOS, XSD

def make_ontology(csv_file, base_url):
    # New graph
    g = Graph()

    # Set up namespaces
    ns = Namespace(base_url)
    g.bind("ns", ns)
    g.bind("skos", SKOS)

    # Make the concept scheme
    scheme = URIRef(ns["EquipmentEfficiencyScheme"])
    g.add((scheme, RDF.type, SKOS.ConceptScheme))
    g.add((scheme, SKOS.prefLabel, Literal("MRF Equipment Efficiency", lang="en")))

    # Grab CSV data
    with open(csv_file, 'r') as file:
        csv_stuff = csv.reader(file)
        headers = next(csv_stuff)[1:]  # Ditch first column
        efficiency_vals = next(csv_stuff)[1:]  # Same here

    # Loop through and make concepts
    for header, efficiency in zip(headers, efficiency_vals):
        # Make URI
        uri = URIRef(ns[header.replace(" ", "_")])

        # Add stuff to graph
        g.add((uri, RDF.type, SKOS.Concept))
        g.add((uri, SKOS.prefLabel, Literal(f"Efficiency of {header.replace('_', ' ')}", lang="en")))
        g.add((uri, SKOS.inScheme, scheme))
        g.add((uri, ns.hasEfficiencyValue, Literal(efficiency, datatype=XSD.float)))

    return g

# Do the thing
csv_file = "corrected_mrf_equipment_efficiency.csv"
base_url = "https://example.org/mrf-equipment/"

ontology = make_ontology(csv_file, base_url)

# Save it
output_file = "mrf_equipment_efficiency_ontology.ttl"
ontology.serialize(destination=output_file, format="turtle")

print(f"Done! Saved to {output_file}")

Done! Saved to mrf_equipment_efficiency_ontology.ttl


In [14]:
import csv
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, SKOS, XSD

def make_ontology(csv_file, base_url):
    # New graph
    g = Graph()

    # Set up namespaces
    units = Namespace("https://vocab.sentier.dev/units/")
    qudt = Namespace("http://qudt.org/schema/qudt/")
    mrf = Namespace(base_url)
    g.bind("units", units)
    g.bind("qudt", qudt)
    g.bind("mrf", mrf)
    g.bind("skos", SKOS)

    # the existing Efficiency concept in chris's thing
    efficiency_qk = URIRef("https://vocab.sentier.dev/units/quantity-kind/Efficiency")

    # make the concept thingy
    scheme = URIRef(base_url)
    g.add((scheme, RDF.type, SKOS.ConceptScheme))
    g.add((scheme, SKOS.prefLabel, Literal("MRF Equipment Efficiency Units", lang="en")))

    # Grab CSV data
    with open(csv_file, 'r') as file:
        csv_stuff = csv.reader(file)
        headers = next(csv_stuff)  # Ditch first column (nevermind) 
        next(csv_stuff)  # Skip efficiency values row

    # Loop through and make concepts
    for header in headers:
        # Make URI
        uri = URIRef(f"{base_url}unit/{header.replace(' ', '_')}")

        # Add stuff to graph
        g.add((uri, RDF.type, SKOS.Concept))
        g.add((uri, SKOS.prefLabel, Literal(f"Efficiency of {header.replace('_', ' ')}", lang="en")))
        g.add((uri, SKOS.inScheme, scheme))
        g.add((uri, qudt.hasQuantityKind, efficiency_qk))
        g.add((uri, SKOS.broader, efficiency_qk))
        g.add((uri, SKOS.notation, Literal(f"{header.replace(' ', '_')}_eff", datatype=qudt.symbol)))
        g.add((uri, SKOS.definition, Literal(f"Ratio of output to input for {header.replace('_', ' ')} in Material Recovery Facility", lang="en")))
        g.add((uri, qudt.latexSymbol, Literal(f"\\eta_{{{header.replace(' ', '_')}}}", datatype=qudt.LatexString)))
        g.add((uri, qudt.informativeReference, Literal("https://en.wikipedia.org/wiki/Material_recovery_facility", datatype=XSD.anyURI)))

    return g

# Do the thing
csv_file = "corrected_mrf_equipment_efficiency.csv"
base_url = "https://vocab.sentier.dev/model-terms/mrf-equipment/"

ontology = make_ontology(csv_file, base_url)

# Save it
output_file = "mrf_equipment_efficiency_ontology.ttl"
ontology.serialize(destination=output_file, format="turtle")

print(f"Done! Saved to {output_file}")

Done! Saved to mrf_equipment_efficiency_ontology.ttl
